In [40]:
import random

# Configurações do mundo
TAMANHO = 5
ORIGEM = (0, 0)
ACOES = ['N', 'S', 'L', 'O', 'pegar', 'escalar']
MOVIMENTOS = {'N': (-1, 0), 'S': (1, 0), 'L': (0, 1), 'O': (0, -1)}

class MundoWumpus:
    def __init__(self):
        self.ouro = self._posicionar_aleatorio(evita=[ORIGEM])
        self.wumpus = self._posicionar_aleatorio(evita=[ORIGEM, self.ouro])
        self.abismos = self._criar_abismos(quantidade=4, evita=[ORIGEM, self.ouro, self.wumpus])
        self.reset()

    def reset(self):
        self.pos = ORIGEM
        self.ouro_pego = False
        self.passos = 0
        self.max_passos = 100

    def _criar_abismos(self, quantidade, evita):
        abismos = set()
        while len(abismos) < quantidade:
            pos = self._posicionar_aleatorio(evita=evita)
            evita.append(pos)
            abismos.add(pos)
        return abismos

    def _posicionar_aleatorio(self, evita=[]):
        while True:
            pos = (random.randint(0, TAMANHO-1), random.randint(0, TAMANHO-1))
            if pos not in evita:
                return pos

    def obter_estado(self):
        return (self.pos[0], self.pos[1], self.ouro_pego)

    def executar_acao(self, acao):
        recompensa = -1
        fim = False

        if acao in MOVIMENTOS:
            dx, dy = MOVIMENTOS[acao]
            nx, ny = self.pos[0] + dx, self.pos[1] + dy

            if 0 <= nx < TAMANHO and 0 <= ny < TAMANHO:
                self.pos = (nx, ny)

            if self.pos == self.wumpus:
                recompensa -= 100
                fim = True

            if self.pos in self.abismos:
                recompensa -= 100
                fim = True

        elif acao == 'pegar':
            if self.pos == self.ouro and not self.ouro_pego:
                self.ouro_pego = True
                recompensa += 50

        elif acao == 'escalar':
            if self.pos == ORIGEM and self.ouro_pego:
                recompensa += 50
                fim = True

        self.passos += 1
        if self.passos >= self.max_passos:
            fim = True

        return recompensa, self.obter_estado(), fim

class AgenteQLearning:
    def __init__(self, ambiente, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.ambiente = ambiente
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def escolher_acao(self, estado):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(ACOES)
        else:
            q_valores = [self.q_table.get((estado, a), 0) for a in ACOES]
            max_q = max(q_valores)
            acoes_max = [a for a, q in zip(ACOES, q_valores) if q == max_q]
            return random.choice(acoes_max)

    def atualizar_Q(self, estado, acao, recompensa, novo_estado):
        chave = (estado, acao)
        q_atual = self.q_table.get(chave, 0)
        max_q_novo = max([self.q_table.get((novo_estado, a), 0) for a in ACOES])
        self.q_table[chave] = q_atual + self.alpha * (recompensa + self.gamma * max_q_novo - q_atual)

def treinar(episodios):
    mundo = MundoWumpus()
    agente = AgenteQLearning(mundo)

    print(f"Ouro está em: {mundo.ouro}")
    print(f"Wumpus está em: {mundo.wumpus}")
    print(f"Abismos estão em: {mundo.abismos}")

    for ep in range(episodios):
        mundo.reset()
        estado = mundo.obter_estado()

        while True:
            acao = agente.escolher_acao(estado)
            recompensa, novo_estado, fim = mundo.executar_acao(acao)
            agente.atualizar_Q(estado, acao, recompensa, novo_estado)
            estado = novo_estado
            if fim:
                break

        if (ep + 1) % 1000 == 0:
            print(f"Episódio {ep+1} finalizado.")

    print("Treinamento concluído.")

    return agente, mundo

def testar(agente, mundo):
    agente.epsilon = 0
    mundo.reset()
    estado = mundo.obter_estado()
    total_recompensa = 0
    trajetoria = []

    while True:
        acao = agente.escolher_acao(estado)
        recompensa, novo_estado, fim = mundo.executar_acao(acao)
        total_recompensa += recompensa
        trajetoria.append((mundo.pos, acao, recompensa, total_recompensa))
        estado = novo_estado
        if fim:
            break

    print("\n--- Teste ---")
    for passo in trajetoria:
        pos, acao, recompensa, acumulado = passo
        print(f"Posição: {pos} | Ação: {acao} | Recompensa: {recompensa} | Acumulado: {acumulado}")

    print(f"\nRecompensa total: {total_recompensa}")

if __name__ == "__main__":
    agente_treinado, mundo = treinar(episodios=10000)
    testar(agente_treinado, mundo)


Ouro está em: (2, 2)
Wumpus está em: (4, 1)
Abismos estão em: {(1, 0), (0, 2), (3, 4), (1, 4)}
Episódio 1000 finalizado.
Episódio 2000 finalizado.
Episódio 3000 finalizado.
Episódio 4000 finalizado.
Episódio 5000 finalizado.
Episódio 6000 finalizado.
Episódio 7000 finalizado.
Episódio 8000 finalizado.
Episódio 9000 finalizado.
Episódio 10000 finalizado.
Treinamento concluído.

--- Teste ---
Posição: (0, 1) | Ação: L | Recompensa: -1 | Acumulado: -1
Posição: (1, 1) | Ação: S | Recompensa: -1 | Acumulado: -2
Posição: (2, 1) | Ação: S | Recompensa: -1 | Acumulado: -3
Posição: (2, 2) | Ação: L | Recompensa: -1 | Acumulado: -4
Posição: (2, 2) | Ação: pegar | Recompensa: 49 | Acumulado: 45
Posição: (2, 1) | Ação: O | Recompensa: -1 | Acumulado: 44
Posição: (1, 1) | Ação: N | Recompensa: -1 | Acumulado: 43
Posição: (0, 1) | Ação: N | Recompensa: -1 | Acumulado: 42
Posição: (0, 0) | Ação: O | Recompensa: -1 | Acumulado: 41
Posição: (0, 0) | Ação: escalar | Recompensa: 49 | Acumulado: 90

Recom

In [37]:
import random

# Configurações do mundo
TAMANHO = 3
ORIGEM = (0, 0)
ACOES = ['N', 'S', 'L', 'O', 'pegar', 'escalar']
MOVIMENTOS = {'N': (-1, 0), 'S': (1, 0), 'L': (0, 1), 'O': (0, -1)}

class MundoWumpus:
    def __init__(self):
        self.ouro = self._posicionar_aleatorio(evita=[ORIGEM])
        self.wumpus = self._posicionar_aleatorio(evita=[ORIGEM, self.ouro])
        self.abismos = self._criar_abismos(quantidade=1, evita=[ORIGEM, self.ouro, self.wumpus])
        self.reset()

    def reset(self):
        self.pos = ORIGEM
        self.ouro_pego = False
        self.passos = 0
        self.max_passos = 100

    def _criar_abismos(self, quantidade, evita):
        abismos = set()
        while len(abismos) < quantidade:
            pos = self._posicionar_aleatorio(evita=evita)
            evita.append(pos)
            abismos.add(pos)
        return abismos

    def _posicionar_aleatorio(self, evita=[]):
        while True:
            pos = (random.randint(0, TAMANHO-1), random.randint(0, TAMANHO-1))
            if pos not in evita:
                return pos

    def obter_estado(self):
        return (self.pos[0], self.pos[1], self.ouro_pego)

    def executar_acao(self, acao):
        recompensa = -1
        fim = False

        if acao in MOVIMENTOS:
            dx, dy = MOVIMENTOS[acao]
            nx, ny = self.pos[0] + dx, self.pos[1] + dy

            if 0 <= nx < TAMANHO and 0 <= ny < TAMANHO:
                self.pos = (nx, ny)

            if self.pos == self.wumpus:
                recompensa -= 100
                fim = True

            if self.pos in self.abismos:
                recompensa -= 100
                fim = True

        elif acao == 'pegar':
            if self.pos == self.ouro and not self.ouro_pego:
                self.ouro_pego = True
                recompensa += 50

        elif acao == 'escalar':
            if self.pos == ORIGEM and self.ouro_pego:
                recompensa += 50
                fim = True

        self.passos += 1
        if self.passos >= self.max_passos:
            fim = True

        return recompensa, self.obter_estado(), fim

class AgenteQLearning:
    def __init__(self, ambiente, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.ambiente = ambiente
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def escolher_acao(self, estado):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(ACOES)
        else:
            q_valores = [self.q_table.get((estado, a), 0) for a in ACOES]
            max_q = max(q_valores)
            acoes_max = [a for a, q in zip(ACOES, q_valores) if q == max_q]
            return random.choice(acoes_max)

    def atualizar_Q(self, estado, acao, recompensa, novo_estado):
        chave = (estado, acao)
        q_atual = self.q_table.get(chave, 0)
        max_q_novo = max([self.q_table.get((novo_estado, a), 0) for a in ACOES])
        self.q_table[chave] = q_atual + self.alpha * (recompensa + self.gamma * max_q_novo - q_atual)

def treinar(episodios):
    mundo = MundoWumpus()
    agente = AgenteQLearning(mundo)

    print(f"Ouro está em: {mundo.ouro}")
    print(f"Wumpus está em: {mundo.wumpus}")
    print(f"Abismos estão em: {mundo.abismos}")

    for ep in range(episodios):
        mundo.reset()
        estado = mundo.obter_estado()

        while True:
            acao = agente.escolher_acao(estado)
            recompensa, novo_estado, fim = mundo.executar_acao(acao)
            agente.atualizar_Q(estado, acao, recompensa, novo_estado)
            estado = novo_estado
            if fim:
                break

        if (ep + 1) % 1000 == 0:
            print(f"Episódio {ep+1} finalizado.")

    print("Treinamento concluído.")

    print("\n--- Q-Table ---")
    for chave in sorted(agente.q_table.keys()):
        estado, acao = chave
        valor_q = agente.q_table[chave]
        print(f"Estado: {estado} | Ação: {acao} | Q-valor: {valor_q:.2f}")


    return agente, mundo

def testar(agente, mundo):
    agente.epsilon = 0
    mundo.reset()
    estado = mundo.obter_estado()
    total_recompensa = 0
    trajetoria = []

    while True:
        acao = agente.escolher_acao(estado)
        recompensa, novo_estado, fim = mundo.executar_acao(acao)
        total_recompensa += recompensa
        trajetoria.append((mundo.pos, acao, recompensa, total_recompensa))
        estado = novo_estado
        if fim:
            break

    print("\n--- Teste ---")
    for passo in trajetoria:
        pos, acao, recompensa, acumulado = passo
        print(f"Posição: {pos} | Ação: {acao} | Recompensa: {recompensa} | Acumulado: {acumulado}")

    print(f"\nRecompensa total: {total_recompensa}")

if __name__ == "__main__":
    agente_treinado, mundo = treinar(episodios=10000)
    testar(agente_treinado, mundo)


Ouro está em: (1, 2)
Wumpus está em: (2, 1)
Abismos estão em: {(1, 0)}
Episódio 1000 finalizado.
Episódio 2000 finalizado.
Episódio 3000 finalizado.
Episódio 4000 finalizado.
Episódio 5000 finalizado.
Episódio 6000 finalizado.
Episódio 7000 finalizado.
Episódio 8000 finalizado.
Episódio 9000 finalizado.
Episódio 10000 finalizado.
Treinamento concluído.

--- Q-Table ---
Estado: (0, 0, False) | Ação: L | Q-valor: 265.60
Estado: (0, 0, False) | Ação: N | Q-valor: 238.04
Estado: (0, 0, False) | Ação: O | Q-valor: 238.04
Estado: (0, 0, False) | Ação: S | Q-valor: -101.00
Estado: (0, 0, False) | Ação: escalar | Q-valor: 238.04
Estado: (0, 0, False) | Ação: pegar | Q-valor: 238.04
Estado: (0, 0, True) | Ação: L | Q-valor: 395.00
Estado: (0, 0, True) | Ação: N | Q-valor: 440.00
Estado: (0, 0, True) | Ação: O | Q-valor: 440.00
Estado: (0, 0, True) | Ação: S | Q-valor: -101.00
Estado: (0, 0, True) | Ação: escalar | Q-valor: 490.00
Estado: (0, 0, True) | Ação: pegar | Q-valor: 440.00
Estado: (0, 